# Решение заданий к уроку 5

In [11]:
import matplotlib.pyplot as plt
import random

from matplotlib.colors import ListedColormap
from sklearn import datasets, model_selection

import numpy as np

__1.__ Сформировать с помощью sklearn.make_classification датасет из 1000 объектов с двумя признаками, обучить случайный лес из 1, 3, 10 и 50, 100, 200 деревьев и визуализировать их разделяющие гиперплоскости на графиках (по подобию визуализации деревьев из предыдущего урока, необходимо только заменить вызов функции predict на tree_vote). Сделать выводы о получаемой сложности гиперплоскости и недообучении или переобучении случайного леса в зависимости от количества деревьев в нем. 2*. Заменить в реализованном алгоритме проверку с помощью отложенной выборки на Out-of-Bag. 3*. (На повторение) Переписать функцию calc_gini из урока про решающие деревья так, чтобы в качестве критерия использовалась энтропия Шэннона. Переименовать функцию в calc_entropy.


### Решение

Возьмем класс DecisionTree из предыдущего урока

In [6]:
# Реализуем класс узла

class Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле

# И класс терминального узла (листа)

class Leaf:
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.prediction = self.predict()
        
    def predict(self):
        # подсчет количества объектов разных классов
        classes = {}  # сформируем словарь "класс: количество объектов"
        for label in self.labels:
            if label not in classes:
                classes[label] = 0
            classes[label] += 1
        #  найдем класс, количество объектов которого будет максимальным в этом листе и вернем его    
        prediction = max(classes, key=classes.get)
        return prediction     

In [7]:
class DecisionTree:
    
    def __init__(self, min_leaf, criterion):
        self.min_leaf = min_leaf
        self.criterion = criterion
    
    # Расчет критерия

    def calc_criterion(self,labels):
        #  подсчет количества объектов разных классов
        classes = {}
        for label in labels:
            if label not in classes:
                classes[label] = 0
            classes[label] += 1

        #  расчет критерия
        if self.criterion == 'gini':
            criter = 1
            for label in classes:
                p = classes[label] / len(labels)
                criter -= p ** 2
#            print('gini', criter)
        elif self.criterion == 'entropy':
            criter = 0
            for label in classes:
                p = classes[label] / len(labels)
                criter -= p * np.log2(p)
#            print('entropy',criter)
        return criter
        
    # Расчет качества
    def quality(self, true_labels, false_labels, criter):        
        # доля выбоки, ушедшая в левое поддерево
        p = float(true_labels.shape[0]) / (true_labels.shape[0] + false_labels.shape[0])
        return criter - p * self.calc_criterion(true_labels) - (1 - p) * self.calc_criterion(false_labels)         
        
    # Разбиение датасета в узле
    def split(self, data, labels, index, t):

        left = np.where(data[:, index] <= t)
        right = np.where(data[:, index] > t)

        true_data = data[left]
        false_data = data[right]
        true_labels = labels[left]
        false_labels = labels[right]

        return true_data, false_data, true_labels, false_labels 
        
    # Нахождение наилучшего разбиения
    def find_best_split(self, data, labels):

        criter = self.calc_criterion(labels)

        best_quality = 0
        best_t = None
        best_index = None
        
        n_features = data.shape[1]
    
        for index in range(n_features):
            # будем проверять только уникальные значения признака, исключая повторения
            t_values = np.unique([row[index] for row in data])

            for t in t_values:
                true_data, false_data, true_labels, false_labels = self.split(data, labels, index, t)
                #  пропускаем разбиения, в которых в узле остается менее 5 объектов
                if len(true_data) < self.min_leaf or len(false_data) < self.min_leaf:
                    continue

                current_quality = self.quality(true_labels, false_labels, criter)

                #  выбираем порог, на котором получается максимальный прирост качества
                if current_quality > best_quality:
                    best_quality, best_t, best_index = current_quality, t, index

        return best_quality, best_t, best_index
        
    # Построение дерева с помощью рекурсивной функции
    def build_tree(self, data, labels): 

        quality, t, index = self.find_best_split(data, labels)        

        #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
        if quality == 0:
            return Leaf(data, labels)

        true_data, false_data, true_labels, false_labels = self.split(data, labels, index, t)        

        # Рекурсивно строим два поддерева
        true_branch = self.build_tree(true_data, true_labels)
        false_branch = self.build_tree(false_data, false_labels)

        # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева        
        return Node(index, t, true_branch, false_branch)
    
    def classify_object(self, obj, node):
        #  Останавливаем рекурсию, если достигли листа
        if isinstance(node, Leaf):
            answer = node.prediction
            return answer

        if obj[node.index] <= node.t:
            return self.classify_object(obj, node.true_branch)
        else:
            return self.classify_object(obj, node.false_branch)

    def fit(self, data, labels):
        self.my_tree = self.build_tree(data, labels)
        
    def predict(self, data):        
        classes = []
        for obj in data:
            prediction = self.classify_object(obj, self.my_tree)        
            classes.append(prediction)
        return classes    

In [8]:
# Введем функцию подсчета точности как доли правильных ответов
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

In [14]:
# Реализуем класс Случайного леса

class RandomForest:
    def __init__(self, min_leaf, criterion, n_trees):
        self.n_trees = n_trees
        self.min_leaf = min_leaf
        self.criterion = criterion        
        self.forest = None

    def get_bootstrap(self, data, labels, N):
        n_samples = data.shape[0]
        bootstrap = []

        for i in range(N):
            b_data = np.zeros(data.shape)
            b_labels = np.zeros(labels.shape)

            for j in range(n_samples):
                sample_index = random.randint(0, n_samples - 1)
                b_data[j] = data[sample_index]
                b_labels[j] = labels[sample_index]
            bootstrap.append((b_data, b_labels))

        return bootstrap        

    def fit(self, data, labels):
        self.forest = []
        bootstrap = self.get_bootstrap(data, labels, self.n_trees)

        for b_data, b_labels in bootstrap:
            tree = DecisionTree(min_leaf = self.min_leaf, criterion=self.criterion)
            tree.fit(b_data, b_labels)
            self.forest.append(tree)

        return self.forest

    def predict(self, data):

        # добавим предсказания всех деревьев в список
        predictions = []

        for tree in self.forest:
            predictions.append(tree.predict(data))

        # сформируем список с предсказаниями для каждого объекта
        predictions_per_object = list(zip(*predictions))

        # выберем в качестве итогового предсказания для каждого объекта то,
        # за которое проголосовало большинство деревьев
        voted_predictions = []
        for obj in predictions_per_object:
            voted_predictions.append(max(set(obj), key=obj.count))

        return voted_predictions

In [12]:
classification_data, classification_labels = datasets.make_classification(n_samples=1000,n_features = 2, n_informative = 2, 
                                                      n_classes = 2, n_redundant=0, 
                                                      n_clusters_per_class=1, random_state=2)
train_data, test_data, train_labels, test_labels = model_selection.train_test_split(classification_data, 
                                                                                     classification_labels, 
                                                                                     test_size = 0.3,
                                                                                     random_state = 1)


In [19]:
tree = RandomForest(min_leaf=5, criterion='gini', n_trees = 50)
tree.fit(train_data, train_labels)
train_answers = tree.predict(train_data)
test_answers = tree.predict(test_data)

In [20]:
print(f'{accuracy_metric(train_labels, train_answers)} \t\t {accuracy_metric(test_labels, test_answers)}')

95.85714285714285 		 91.0


In [21]:
print(f'n_trees\t\t train_accuracy \t\t  test_accuracy')
for n in [1,3,10,50,100]:
    tree = RandomForest(min_leaf=5, criterion= 'entropy', n_trees = n)
    tree.fit(train_data, train_labels)
    train_answers = tree.predict(train_data)
    test_answers = tree.predict(test_data)
    print(f'{n}\t\t {accuracy_metric(train_labels, train_answers)} \t\t {accuracy_metric(test_labels, test_answers)}')

n_trees		 train_accuracy 		  test_accuracy
1		 91.71428571428571 		 86.66666666666667
3		 94.57142857142857 		 89.33333333333333
10		 95.42857142857143 		 90.33333333333333
50		 96.0 		 91.0
100		 96.57142857142857 		 91.66666666666666
